# Getting Started : Optimal Transport Between Two Datasets

This tutorial covers the most basic use of the `ott` toolbox:

- Compute the optimal transport between two point clouds using the {class}`~ott.solvers.linear.sinkhorn.Sinkhorn` algorithm. 
- Use the seamless integration with `jax` to differentiate through this operation.

## Imports and toy data definition

`ott` is built on top of `jax`, so we use `jax` to define all the variables of the problem.

In [ ]:
import jax
import jax.numpy as jnp

We also import `matplotlib` to plot the results.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cycler

mpl.rcParams["axes.prop_cycle"] = cycler(
    color=[
        "#377eb8",
        "#ff7f00",
        "#4daf4a",
        "#f781bf",
        "#a65628",
        "#984ea3",
        "#999999",
        "#e41a1c",
        "#dede00",
    ]
)  # colorblind friendly cycle from https://gist.github.com/thriveth/8560036

We define a toy function and use it to generate two datasets of points, `X_target` and `X_source`:

In [ ]:
def create_points(rng, n_samples_source, n_samples_target, n_features):
    rngs = jax.random.split(rng, 2)
    X_source = jax.random.uniform(rngs[1], (n_samples_source, n_features))
    X_target = jax.random.normal(rngs[0], (n_samples_target, n_features)) + 1.0
    return X_source, X_target


rng = jax.random.PRNGKey(0)
n_samples_source, n_samples_target, n_features = 13, 20, 2
X_source, X_target = create_points(
    rng, n_samples_source, n_samples_target, n_features
)

plt.scatter(X_source[:, 0], X_source[:, 1], label="source")
plt.scatter(X_target[:, 0], X_target[:, 1], label="target", marker="x")
plt.legend()

## Optimal transport with OTT

We will now use `ott` to compute the optimal transport between `X_target` and `X_source`. To do so, we first create a `geometry` object that defines the geometry between the two point clouds:

In [ ]:
from ott.geometry import pointcloud

geometry = pointcloud.PointCloud(X_target, X_source)

`geometry` contains the two datasets `X_target` and `X_source`, as well as a `cost_fn` that is a way to measure distances between points. Here, we use the default settings, so the `cost_fn` is {class}`~ott.geometry.costs.SqEuclidean`, the standard squared Euclidean distance.

In order to compute the optimal transport corresponding to this `geometry`, we use the Sinkhorn algorithm. The Sinkhorn algorithm has an hyperparameter `epsilon` that controls the amount of regularization. It is stored in `geometry`, and by default it is 1/20th of the mean cost value.

In [ ]:
from ott.solvers.linear import sinkhorn

ot = sinkhorn.solve(geometry)

As a small note: the computations here are *jitted*, meaning that the second time the solver is run it will be much faster:

In [ ]:
ot = sinkhorn.solve(geometry)

This object `ot` contains all the information about the solution of the optimal transport problems, including the transport plan matrix that indicates how much of the mass of the point `X_target[i]` is moved towards `X_source[j]`.

In [ ]:
plt.imshow(ot.matrix)
plt.colorbar()

## Seamless integration in JAX automatic differentiation pipelines

We finish this quick tour by illustrating one of the most interesting feature of `ott`: it can be plugged easily into differentiable pipelines built in `jax`, like neural networks.

We demonstrate this by differentiating the loss of optimal transport with respect to the input dataset.

To do so, we simply define a function that takes `X_source` as input and outputs the loss:

In [ ]:
def cost_from_source(X_source):
    geometry = pointcloud.PointCloud(X_target, X_source)
    ot = sinkhorn.solve(geometry)
    return ot.reg_ot_cost


cost_from_source(X_source)

The gradient of this function is obtained as usual with `jax`:

In [ ]:
gradient_function = jax.grad(cost_from_source)
gradient = gradient_function(X_source)

This gradient defines a direction for each sample in `X_source`, and following the opposite of this direction leads to a decrease in the primal cost: it pushes `X_source` closer to `X_target`.

In [ ]:
plt.scatter(X_source[:, 0], X_source[:, 1], label="source")
plt.quiver(X_source[:, 0], X_source[:, 1], -gradient[:, 0], -gradient[:, 1])
plt.scatter(X_target[:, 0], X_target[:, 1], label="target", marker="x")
plt.legend()

For instance, this gradient can be used to update the dataset `X_source` to make it go towards `X_target`:

In [ ]:
step_size = 2.0

f, axes = plt.subplots(1, 4, sharey=True, sharex=True, figsize=(12, 3))
for iteration, axe in enumerate(axes):
    gradient = gradient_function(X_source)
    axe.scatter(X_source[:, 0], X_source[:, 1], label=f"source")
    axe.quiver(X_source[:, 0], X_source[:, 1], -gradient[:, 0], -gradient[:, 1])
    axe.scatter(X_target[:, 0], X_target[:, 1], label="target", marker="x")
    axe.set_title(f"iteration {iteration}")
    X_source -= step_size * gradient
